# Init

In [1]:
# ----------- set params ----------- #

writefiles = True
render_maps = False

# buffer around input perimeter polygon
perimeter_buffer = 10000

crs_global = 4326
crs_dem = 25832

In [2]:
# ----------- init -----------
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
from datetime import date
import folium
from shapely.geometry import LineString, MultiLineString
import plotly.graph_objects as go
from collections import defaultdict
from datetime import datetime
from shapely import wkb
from shapely.geometry import Point
import rasterio
from rasterio.merge import merge
import numpy as np
from pathlib import Path
from shapely.geometry import box
import geopandas as gpd
import numpy as np


# custom skip magic
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    pass  # Ignores execution



# ----------- attributes to retrieve -----------
ox.settings.useful_tags_way = [
    'highway', 'lanes', 
    'surface', 'lit', 'maxspeed', 'landuse', 'junction',
    'oneway', 'oneway:bicycle', 'bicycle', 
    'cycleway',  
    
    'cycleway:right', 'cycleway:left', 'cycleway:both',
    'cycleway:right:oneway', 'cycleway:left:oneway',

]

ox.settings.useful_tags_node = [
    'asl', 'bicycle_parking', 'cycleway'
]

In [4]:
cycle_edges = gpd.read_file(r'\\tsclient\O\Public\4233-111411-MobiJoule\uelii\01_data\network_copenhagen_250522\osm_edges_MobiJoule_2025-05-22.gpkg')

In [5]:
# ----------- additions through corin : booleans -----------

cycle_edges['highway_is_pedestrian'] = ((cycle_edges['highway'] == 'pedestrian') | 
                                        (cycle_edges['highway'] == 'footway') | 
                                        (cycle_edges['highway'] == 'path')                                       
                                        )

cycle_edges['cycletrack_assumed_separate'] = (
    ((cycle_edges['cycleway:right'] == 'separate') & (~cycle_edges['reversed'])) |
    ((cycle_edges['cycleway:left'] == 'separate') & (cycle_edges['reversed']))
)

# Write

In [6]:
gpd.options.io_engine = "pyogrio"

# -----------------------------------------------------------------------------
# ---- write files ----
# -----------------------------------------------------------------------------

if writefiles:
        
    #today_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    today_str = datetime.now().strftime("%Y-%m-%d")
    
    cycle_edges_out_extended = cycle_edges.to_crs(32632)
    cycle_edges_out_short = cycle_edges_out_extended[["network_id", "highway", "cycleway_master", "geometry"]]

    # ----------- parquet ----------- # 
    #cycle_edges_out_extended.to_parquet(f"out/osm_nodes_MobiJoule_{today_str}.parquet")
    #cycle_edges_out_short.to_parquet(f"out/osm_edges_MobiJoule_{today_str}_short.parquet")

    # ----------- gpkg ----------- # 
    cycle_edges_out_extended.to_file(f"out/osm_edges_MobiJoule_{today_str}.gpkg", layer='cycle_edges', driver="GPKG")
    #cycle_edges_out_short.to_file(f"out/osm_edges_MobiJoule_{today_str}_short.gpkg", layer='cycle_edges', driver="GPKG")
    
        
    # csv
    #cycle_edges_csv = cycle_edges.drop(columns = "geometry")
    #cycle_edges_csv.to_csv(f"out/osm_edges_MobiJoule_{today_str}.csv", index=False)


In [ ]:
sys.exit()

In [ ]:
# ----------- plot intersection clusters -----------

import folium
import geopandas as gpd
import random
import matplotlib.colors as mcolors
import itertools


# Filter out rows with null node_cluster and convert CRS
gdf_plot = cycle_nodes_assigned[cycle_nodes_assigned['node_cluster'].notna()].to_crs(4326)

# Ensure cluster values are strings

# Assign colors
base_colors = list(mcolors.CSS4_COLORS.values())
color_cycle = itertools.cycle(base_colors)
clusters = gdf_plot['node_cluster'].unique()

cluster_colors = {
    cluster: 'lightgrey' if cluster == -1 else next(color_cycle)
    for cluster in clusters
}


# Center map on data
center = gdf_plot.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=12)

# Add points to map with color and tooltip
for _, row in gdf_plot.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color=cluster_colors[row['node_cluster']],
        fill=False,
        fill_opacity=0.8,
        tooltip=f"Cluster: {row['node_cluster']}"
    ).add_to(m)

m



In [ ]:
# ----------- plot altitudes -----------

import folium
import geopandas as gpd
from folium import CircleMarker

gdf_plot = gdf[pd.notna(gdf['altitude_dem'])].to_crs(crs_global)
# Normalize altitude values for color mapping
min_alt = gdf_plot['altitude_dem'].min()
max_alt = gdf_plot['altitude_dem'].max()
gdf_plot['altitude_norm'] = (gdf_plot['altitude_dem'] - min_alt) / (max_alt - min_alt)

# Define a function to convert normalized altitude to color (blue to red gradient)
def altitude_to_color(norm_val):
    r = int(255 * norm_val)
    b = int(255 * (1 - norm_val))
    return f'#{r:02x}00{b:02x}'

# Create folium map centered on the data
center = [gdf_plot.geometry.y.mean(), gdf_plot.geometry.x.mean()]
m = folium.Map(location=center, zoom_start=10)

# Add points
for _, row in gdf_plot.iterrows():
    color = altitude_to_color(row['altitude_norm'])
    CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.8,
        tooltip=str(row['altitude_dem'])

    ).add_to(m)

m